In [1]:
import sys  
sys.path.append("./src")

import pandas as pd
import numpy as np
import time
from gym_trading_env.environments import TradingEnv
import gymnasium as gym
from stable_baselines3 import PPO
from gym_trading_env.renderer import Renderer


df = pd.read_csv(r"C:\Users\user\Documents\GitHub\Gym-Trading-Env\examples\data\pistar\SS00001.csv", index_col=0)
df.dropna(inplace=True)

df['feature_close'] = (df['close'] - df['close'].min()) / (df['close'].max() - df['close'].min())
df["feature_volume"] = df['volume'].copy()
df["feature_RSI"] = df["RSI"].copy()
df["feature_MACD"] = df["MACD"].copy()
df["feature_CCI"] = df["CCI"].copy()
df["feature_ADX"] = df["ADX"].copy()

 
# Generating features
df.dropna(inplace= True)

# Create your own reward function with the history object
def log_return_reward_function(history):
    return np.log(history["portfolio_valuation", -1] / history["portfolio_valuation", -2])

def paper_reward_function(history):
    return (history["portfolio_valuation", -1] - history["portfolio_valuation", -2]) / history["portfolio_valuation", -2]


env = gym.make(
        "TradingEnv",
        name= "stock",
        df = df,
        windows= 1,
        positions = [0, 0.5, 1, 1.5, 2], # From -1 (=SHORT), to +1 (=LONG)
        initial_position = 'random', #Initial position
        trading_fees = 0.01/100, # 0.01% per stock buy / sell
        fiat_borrow_interest_rate = 0,
        asset_borrow_interest_rate= 0,
        reward_function = paper_reward_function,
        portfolio_initial_value = 1000, # in FIAT (here, USD)
        max_episode_duration = 'max',
    )

done, truncated = False, False
observation, info = env.reset()
print(info)

model = PPO()

while not done and not truncated:
    action, _states = model.predict(observation)
    observation, reward, done, truncated, info = env.step(action)


# Save for render
env.unwrapped.save_for_render()

# Rendering
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13